# Indeed webscraper

page 1= https://nl.indeed.com/jobs?q=data+analist&start=00  <br>
page 2= https://nl.indeed.com/jobs?q=data+analist&start=10 <br>
page 3= https://nl.indeed.com/jobs?q=data+analist&start=20 <br>


**Getting the link of the job**:

`<table id="resultsBody"> --> <td id="resultsCol"> --> <a (containing href=)>` --> href == link to job

**Getting job description**:
`<div id="jobDescriptionText"> --> <p> of (<ul> --> <li>)` --> paragraphs of text


## Load libraries

In [1]:
library(tidyverse)
library(rvest)
library(XML)
library(stringr)
library(lubridate)

-- Attaching packages --------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.6     v dplyr   1.0.8
v tidyr   1.2.0     v stringr 1.4.0
v readr   2.1.2     v forcats 0.5.1

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


Attaching package: 'rvest'


The following object is masked from 'package:readr':

    guess_encoding



Attaching package: 'lubridate'


The following objects are masked from 'package:base':

    date, intersect, setdiff, union




In [2]:
job_base_url <- 'https://nl.indeed.com'
job_base_search_url <- 'https://nl.indeed.com/jobs?q=data&start='
# url_job_search <- read_html('https://nl.indeed.com/jobs?q=data')

wordlist <- read.table(
    '..\\Data_cleaned\\D001\\trimmed_skill_names.txt',
     header = TRUE
     ) %>% .$x

jobs_df <- data.frame(
    Company = character(),
    job_title = character(),
    salary = character(),
    skills = character(),
    days_online = numeric(),
    job_desc = character(),
    job_link = character()
)

# gather ~330 job listings
for (i in seq.default(0, 290, 10)){

    job_list_url <- paste(job_base_search_url, as.character(i), sep= "")
    job_list_read <- read_html(job_list_url) # lees html in

    job_urls <- job_list_read %>%
        html_nodes(xpath= '//*[@id="mosaic-zone-jobcards"]') %>% # ga naar het specifieke xpath
        html_nodes('a') %>% # ga naar a
        html_attr("href")# pak de href

    # loop over job_urls
    for (j in 1:length(job_urls)){

        tryCatch(
            # Try below code
            expr = {
                # when url is longer than 200 chars it is a job url
                # nchar(job_urls[[j]]) > 200
                if (grepl('/pagead/', job_urls[[j]]) | grepl('/clk?', job_urls[[j]]) | grepl('/company/', job_urls[[j]])  ){
                    full_job_url <- paste(job_base_url, job_urls[[j]], sep="")
                    
                    # read html
                    job_html_read <- read_html(full_job_url)

                    # extract company name
                    company_name <- job_html_read %>%
                        html_nodes(css= 'head > meta:nth-child(15)') %>%
                        html_attr('content')

                    # extract title
                    job_title <- job_html_read %>%
                        html_node('h1') %>%
                        html_text()
                    
                    time_since_placed_text <- job_html_read %>%
                        html_nodes(css= '.jobsearch-JobMetadataFooter > div:nth-child(2)') %>%
                        html_text()

                    # check if string does not contain integer using regex
                    if (!grepl("\\d", time_since_placed_text) & !grepl("Vandaag", time_since_placed_text)){
                        time_since_placed_text <- job_html_read %>%
                            html_nodes(css= '.jobsearch-JobMetadataFooter > div:nth-child(1)') %>%
                            html_text()
                    }

                    if (grepl("\\d", time_since_placed_text)){
                        time_since_placed <- str_extract_all(time_since_placed_text, "\\(?[0-9,.]+\\)?")[[1]]
                    } else if (grepl("Vandaag", time_since_placed_text)) {
                        time_since_placed <- 0
                    } else {
                    time_since_placed <- NA
                    }


                    # extract description
                    job_description <- job_html_read %>%
                        html_nodes(xpath= '//*[@id="jobDescriptionText"]') %>%
                        html_text()
                    
                    # remove non alpha numeric characters
                    job_description_cleaned <- gsub('[^[:alnum:] ]', ' ', job_description)

                    # generate wordvector
                    wordvec <- unname(unlist(sapply(job_description_cleaned, function(z) str_split(tolower(z), " "))))

                    # extract skills from word vector
                    skill_table <- wordvec[wordvec %in% tolower(wordlist)] %>%
                        unique()

                    # probeer salari op te halen via specifieke salary indicator
                    salary_indicator <- job_html_read %>%
                        html_nodes(css= '#salaryInfoAndJobType') %>%
                        html_text()

                    # Als de € niet voorkomt in het bestand probeer dan de salary distribution
                    if (length(salary_indicator) < 1){
                        salary_indicator <- job_html_read %>%
                            html_nodes(css= '.cmp-SalaryDistributionDisplayWidget-subinfo') %>%
                            html_text()
                    }

                    if (length(salary_indicator) < 1){
                        salary <- NA
                    } else if (grepl('€',salary_indicator)) {
                        # anders zoek de eerste salaris indicatie
                        salary_start_num <- unlist(gregexpr('€', salary_indicator))[1]
                        salary <- substring(salary_indicator, salary_start_num, salary_start_num + 7) 
                    } else {
                        salary <- NA
                    }

                    new_row <- c(
                        company_name,
                        job_title,
                        salary,
                        toString(unique(skill_table)),
                        as.numeric(time_since_placed),
                        job_description_cleaned,
                        full_job_url)
        
                    jobs_df[nrow(jobs_df) + 1,] = new_row
                }
            },
            warning = function(w){
                message("a warning occured with url. ", full_job_url)
                message("the data trying to be stored in the the dataframe is:")
                message(new_row)
                message("And below is the warning message from R:")
                message(w)

            },
            error=function(error_message) {
                message("an error occured with url.", full_job_url)
                message("And below is the error message from R:")
                message(error_message)
            },
            finally = {
                Sys.sleep(4)
            }
        )
    }
}

an error occured with url.https://nl.indeed.com/rc/clk?jk=8e5929846b892d61&fccid=9693bb0696b7a117&vjs=3

And below is the error message from R:



ERROR: Error in open.connection(x, "rb"): HTTP error 500.


In [3]:
jobs_df$listing_date <- today() - as.numeric(jobs_df$days_online)

In [4]:
file_n<-sprintf("..\\Data_raw\\D002\\% s scraped indeed data.csv", today())
write.csv(jobs_df, file_n)

## getting indeed search page information
base url= https://nl.indeed.com/jobs?q=data+analist

In [ ]:
url_job_search <- read_html('https://nl.indeed.com/jobs?q=data') # lees html in

job_urls <- url_job_search %>%
  # html_nodes('body')%>%
  html_nodes(xpath= '//*[@id="mosaic-zone-jobcards"]') %>% # ga naar het specifieke xpath
  html_nodes('a') %>% # ga naar a
  html_attr("href")# pak de href

job_urls[[1]]

## Get Job info

In [ ]:
job_base_url <- 'https://nl.indeed.com'
full_job_url <- paste(job_base_url, job_urls[[1]], sep="")
full_job_url

In [ ]:
job_description <- read_html(full_job_url) %>%
    html_nodes(xpath= '//*[@id="jobDescriptionText"]') %>%
    html_text()

## Extract Salary

In [ ]:
test_url = 'https://nl.indeed.com/viewjob?jk=33d4be78835757d1&tk=1ft2v35nf3b5m000&from=serp&vjs=3&advn=4610837443530944&adid=371890778&ad=-6NYlbfkN0A_cWh5ufxJCEatf9FOtgdwl-LajcKrnY3NwdklpeSDnjaH9EsBM6ngV0apa0_Kx2a5GKCW4uS0O7MdMAqsdKUS4q0L_C77bzQCZHbh2VakwHOoBSx_gXNPBJOeqxq7IUUc4s9CBf_lt5ktVCCKOckMPI8DQ-kKHnVfI97NdL-7of8Oo7OCSaCBZL8Z41xkIGi-jjU-emgJ8f1utRtU276rmh-6Rww5rOQKaWr9wwgGy6sYrcY4MEIcYBkbm5BychuYGL0AFbd7Vy65yEMtdbRprQHT9KeUwpjRNpt6FLJMpPGJcxEUzshqpQrMGS0B_CRCLlmxqTWqBEPfpy3Y7aujfdqxmlVIkXqhghqlEzA3Ag==&sjdu=0ZFwD5rbjMRcHz87Kzx_g4iztdTb_2bOBnSpyL0eJqrnoH0zhNHGeUS0d52A-25mtSPp3ZGAbkfQnEhseHJ_YqiTRf1RGyCMrfm3GKjpLJXmRb1As35_i94VtIfO7AZtgI0QXqDAYweserYyqDu1JUW2c6S_FT2kfNZmCatNflk'

salary_indicator <- read_html(test_url) %>%
    html_nodes(css= '.cmp-SalaryDistributionDisplayWidget-subinfo') %>%
    html_text()

salary_indicator

# /html/head/meta[12]
# /html/head/meta[13]
#html.js-focus-visible head meta
#/html/head/meta[13]
#head > meta:nth-child(15)
# head > meta:nth-child(14)

In [ ]:
test_url = 'https://nl.indeed.com/viewjob?jk=aee5f00cab6957ed&q=SAS&tk=1ft62l0v1i9b7802&from=hpd&advn=1272578650180431&adid=367702106&ad=-6NYlbfkN0Bxu6CKaaZeHzJ6IiDpuxwlJ-Jh_LThrAM6mK1WPabT4wOrE7tVDSerQupuoNaldqveRRnZCn-I_tIp5wyi9ean77lpFwBWstwxWV6ehvvWPTacCGfEVwci_XNF1I6UiLnkRlxMaZxMJ1A0xzl8P6ndKdEQCOLPjfz7mv7LBNSdXAwbuEBBwSRXujtr445DebYvoFEty_c9Vrt6wdaxxEfJ2ve64b_DHepatrE8CzLXyC-Qy_cPPLSSFsU32boQAAWRX79YQJhOtj4yzea5N11zenTDua9hFlmQN427rfZLBZHFobUmR28-AIP-Hld2yuVYYFFeREW8oSPbaicTI_TD9R9NNequqrTC88g6KiCW9F-VM18wDDRihihTtZx9-s4TgWF38xVy56wWpuRxuL3STAAyakfmT9_XxKQGpMRR7Q%3D%3D&pub=4a1b367933fd867b19b072952f68dceb&vjs=3'

salary_indicator <- read_html(test_url) %>%
    html_nodes(css= '#salaryInfoAndJobType') %>%
    html_text()


salary_indicator

In [ ]:
# probeer salaris op 1 van de 2 manieren te bepalen
test_url = 'https://nl.indeed.com/viewjob?jk=aee5f00cab6957ed&q=SAS&tk=1ft62l0v1i9b7802&from=hpd&advn=1272578650180431&adid=367702106&ad=-6NYlbfkN0Bxu6CKaaZeHzJ6IiDpuxwlJ-Jh_LThrAM6mK1WPabT4wOrE7tVDSerQupuoNaldqveRRnZCn-I_tIp5wyi9ean77lpFwBWstwxWV6ehvvWPTacCGfEVwci_XNF1I6UiLnkRlxMaZxMJ1A0xzl8P6ndKdEQCOLPjfz7mv7LBNSdXAwbuEBBwSRXujtr445DebYvoFEty_c9Vrt6wdaxxEfJ2ve64b_DHepatrE8CzLXyC-Qy_cPPLSSFsU32boQAAWRX79YQJhOtj4yzea5N11zenTDua9hFlmQN427rfZLBZHFobUmR28-AIP-Hld2yuVYYFFeREW8oSPbaicTI_TD9R9NNequqrTC88g6KiCW9F-VM18wDDRihihTtZx9-s4TgWF38xVy56wWpuRxuL3STAAyakfmT9_XxKQGpMRR7Q%3D%3D&pub=4a1b367933fd867b19b072952f68dceb&vjs=3'

# probeer eerst op te halen via specifieke salary indicator
salary_indicator <- read_html(test_url) %>%
    html_nodes(css= '#salaryInfoAndJobType') %>%
    html_text()


# Als de € niet voorkomt in het bestand probeer dan de salary distribution
if (!grepl( "€", salary_indicator, fixed = TRUE)){
    salary_indicator <- read_html(test_url) %>%
        html_nodes(css= '.cmp-SalaryDistributionDisplayWidget-subinfo') %>%
        html_text()
}

# Als nu de salary_indicator lengte 0 heeft vul de waarden na met NA
if (!length(salary_indicator)){
    salary <- NA
} else {
    salary_start_num <- unlist(gregexpr('€', salary_indicator))[1]
    salary_end_num <- unlist(gregexpr(' ', salary_indicator))[1]

    salary <- substring(salary_indicator, salary_start_num, salary_end_num)  
}

salary_indicator
salary


## remove characters from vector

In [ ]:
job_description

job_description_cleaned <- gsub('[^[:alnum:] ]', ' ', job_description)

job_description_cleaned

word_vec <- unname(unlist(sapply(job_description_cleaned, function(z) str_split(tolower(z), " "))))

word_vec

In [ ]:
wordlist <- c("SQL", "Python", ".NET", "noSQL", "sneakers", 'jij')
test_table <- word_vec[word_vec %in% tolower(wordlist)]
test_table
unique(test_table)
data.frame(x = toString(unique(test_table)))


## Get time since placement

In [ ]:
library(stringr)
test_url = 'https://nl.indeed.com/viewjob?jk=33d4be78835757d1&tk=1ft2v35nf3b5m000&from=serp&vjs=3&advn=4610837443530944&adid=371890778&ad=-6NYlbfkN0A_cWh5ufxJCEatf9FOtgdwl-LajcKrnY3NwdklpeSDnjaH9EsBM6ngV0apa0_Kx2a5GKCW4uS0O7MdMAqsdKUS4q0L_C77bzQCZHbh2VakwHOoBSx_gXNPBJOeqxq7IUUc4s9CBf_lt5ktVCCKOckMPI8DQ-kKHnVfI97NdL-7of8Oo7OCSaCBZL8Z41xkIGi-jjU-emgJ8f1utRtU276rmh-6Rww5rOQKaWr9wwgGy6sYrcY4MEIcYBkbm5BychuYGL0AFbd7Vy65yEMtdbRprQHT9KeUwpjRNpt6FLJMpPGJcxEUzshqpQrMGS0B_CRCLlmxqTWqBEPfpy3Y7aujfdqxmlVIkXqhghqlEzA3Ag==&sjdu=0ZFwD5rbjMRcHz87Kzx_g4iztdTb_2bOBnSpyL0eJqrnoH0zhNHGeUS0d52A-25mtSPp3ZGAbkfQnEhseHJ_YqiTRf1RGyCMrfm3GKjpLJXmRb1As35_i94VtIfO7AZtgI0QXqDAYweserYyqDu1JUW2c6S_FT2kfNZmCatNflk'
test_url2 = 'https://nl.indeed.com/viewjob?jk=a4f0bcd239475ac5&q=data+analist&tk=1ftanb2kisu17800&from=web&advn=9087037695885843&adid=383523449&ad=-6NYlbfkN0D_XjDCAAdMmk6duX_ctb43y51tBCQMBYqAQgUo9Z3gKkSSAcm5qB6MEluEyQCzZ4Qvktb_X2s0ncEMl2dcKSdOuPOzHeSOf6ZwGIA7KGgoH4fSW1JFduihoiyO8TBp2nKKYBEHE7iAi4KAZ2m6AiqwzrmuN3eJma-6mwzCf030BjJnuXNyZrUgc66Jy0ljyxBBJwVnO20F0l0IQ2F42VdP4HMPZB643QzoQbUTllGRYOw9axf9q1WCiqw1DcpbA0wJ6mrqXnCm1SuydyEQ0eIbcOblRKZj8OKDQ29FSs8YWeKq7m-fOy7CSlIAtAQfaOZmkEV6nilpTrISaNlsI0anpaZOvRqojcG9fLWfubbWHopLAhwris69mgtY5tnBSvM%3D&pub=4a1b367933fd867b19b072952f68dceb&vjs=3'

time_since_placed_text <- read_html(test_url) %>%
    html_nodes(css= '.jobsearch-JobMetadataFooter > div:nth-child(2)') %>%
    html_text()

# check if string does not contain integer using regex
if (!grepl("\\d", time_since_placed_text)){
    time_since_placed_text <- read_html(test_url2) %>%
        html_nodes(css= '.jobsearch-JobMetadataFooter > div:nth-child(1)') %>%
        html_text()
}

time_since_placed <- str_extract_all(time_since_placed_text, "\\(?[0-9,.]+\\)?")[[1]] %>%
    as.numeric()

time_since_placed


# Get placement date

In [ ]:
library(lubridate)
time_since_placed
test_date <- today() - time_since_placed
class(test_date)
class(today())
class(time_since_placed)
class(today() - time_since_placed)
test <-today()
class(test)

# Debugging

In [1]:
library(tidyverse)
library(rvest)
library(XML)
library(stringr)
library(lubridate)

-- Attaching packages --------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.6     v dplyr   1.0.8
v tidyr   1.2.0     v stringr 1.4.0
v readr   2.1.2     v forcats 0.5.1

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


Attaching package: 'rvest'


The following object is masked from 'package:readr':

    guess_encoding



Attaching package: 'lubridate'


The following objects are masked from 'package:base':

    date, intersect, setdiff, union




In [10]:
job_base_url <- 'https://nl.indeed.com'
job_base_search_url <- 'https://nl.indeed.com/jobs?q=data&start='
# url_job_search <- read_html('https://nl.indeed.com/jobs?q=data')
test= "https://nl.indeed.com/rc/clk?jk=1fb32f9ae25aaced&fccid=38c84708b80970a7&vjs=3"

wordlist <- read.table(
    '..\\Capstone\\trimmed_skill_names.txt',
     header = TRUE
     ) %>% .$x

jobs_df <- data.frame(
    Company = character(),
    job_title = character(),
    salary = character(),
    skills = character(),
    days_online = numeric(),
    job_desc = character(),
    job_link = character()
)

tryCatch(
    # Try below code
    expr = {
        # when url is longer than 200 chars it is a job url
        # # nchar(job_urls[[j]]) > 200
        # if (grepl('/pagead/', job_urls[[j]]) | grepl('/clk?', job_urls[[j]]) | grepl('/company/', job_urls[[j]])  ){
        #     full_job_url <- paste(job_base_url, job_urls[[j]], sep="")
            
            # read html
            job_html_read <- read_html(test)

            # extract company name
            company_name <- job_html_read %>%
                html_nodes(css= 'head > meta:nth-child(15)') %>%
                html_attr('content')

            # extract title
            job_title <- job_html_read %>%
                html_node('h1') %>%
                html_text()
            
            time_since_placed_text <- job_html_read %>%
                html_nodes(css= '.jobsearch-JobMetadataFooter > div:nth-child(2)') %>%
                html_text()

            # check if string does not contain integer using regex
            if (!grepl("\\d", time_since_placed_text) & !grepl("Vandaag", time_since_placed_text)){
                time_since_placed_text <- job_html_read %>%
                    html_nodes(css= '.jobsearch-JobMetadataFooter > div:nth-child(1)') %>%
                    html_text()
            }

            if (grepl("\\d", time_since_placed_text)){
                time_since_placed <- str_extract_all(time_since_placed_text, "\\(?[0-9,.]+\\)?")[[1]]
            } else if (grepl("Vandaag", time_since_placed_text)) {
                time_since_placed <- 0
            } else {
            time_since_placed <- NA
            }


            # extract description
            job_description <- job_html_read %>%
                html_nodes(xpath= '//*[@id="jobDescriptionText"]') %>%
                html_text()
            
            # remove non alpha numeric characters
            job_description_cleaned <- gsub('[^[:alnum:] ]', ' ', job_description)

            # generate wordvector
            wordvec <- unname(unlist(sapply(job_description_cleaned, function(z) str_split(tolower(z), " "))))

            # extract skills from word vector
            skill_table <- wordvec[wordvec %in% tolower(wordlist)] %>%
                unique()

            # probeer salari op te halen via specifieke salary indicator
            salary_indicator <- job_html_read %>%
                html_nodes(css= '#salaryInfoAndJobType') %>%
                html_text()

            # Als de € niet voorkomt in het bestand probeer dan de salary distribution
            if (length(salary_indicator) < 1){
                salary_indicator <- job_html_read %>%
                    html_nodes(css= '.cmp-SalaryDistributionDisplayWidget-subinfo') %>%
                    html_text()
            }

            if (length(salary_indicator) < 1){
                salary <- NA
            } else if (grepl('€',salary_indicator)) {
                # anders zoek de eerste salaris indicatie
                salary_start_num <- unlist(gregexpr('€', salary_indicator))[1]
                salary <- substring(salary_indicator, salary_start_num, salary_start_num + 7) 
            } else {
                salary <- NA
            }
            new_row <- c(
                company_name,
                job_title,
                salary,
                toString(unique(skill_table)),
                as.numeric(time_since_placed),
                job_description_cleaned,
                full_job_url)
        
            jobs_df[nrow(jobs_df) + 1,] = new_row
        },
    warning = function(w){
        message("a warning occured with url. ", full_job_url)
        message("the data trying to be stored in the the dataframe is:")
        message(new_row[1])
        message(new_row[2])
        message(new_row[3])
        message(new_row[4])
        message(new_row[5])
        message(new_row[6])
        message(new_row[7])
        message(new_row[8])
        message("And below is the warning message from R:")
        message(w)

    },
    error=function(error_message) {
        message("an error occured with url.", full_job_url)
        message("And below is the error message from R:")
        message(error_message)
    },
    finally = {
        Sys.sleep(4)
    }
)



In [12]:
job_base_url <- 'https://nl.indeed.com'
job_base_search_url <- 'https://nl.indeed.com/jobs?q=data&start='
# url_job_search <- read_html('https://nl.indeed.com/jobs?q=data')

wordlist <- read.table(
    '..\\Capstone\\trimmed_skill_names.txt',
     header = TRUE
     ) %>% .$x

jobs_df <- data.frame(
    Company = character(),
    job_title = character(),
    salary = character(),
    skills = character(),
    days_online = numeric(),
    job_desc = character(),
    job_link = character()
)

# gather ~300 - 450 job listings
for (i in seq.default(0, 10, 10)){

    job_list_url <- paste(job_base_search_url, as.character(i), sep= "")
    job_list_read <- read_html(job_list_url) # lees html in

    job_urls <- job_list_read %>%
        html_nodes(xpath= '//*[@id="mosaic-zone-jobcards"]') %>% # ga naar het specifieke xpath
        html_nodes('a') %>% # ga naar a
        html_attr("href")# pak de href

    # loop over job_urls
    for (j in 1:length(job_urls)){

        tryCatch(
            # Try below code
            expr = {
                # when url is longer than 200 chars it is a job url
                # nchar(job_urls[[j]]) > 200
                if (grepl('/pagead/', job_urls[[j]]) | grepl('/clk?', job_urls[[j]]) | grepl('/company/', job_urls[[j]])  ){
                    full_job_url <- paste(job_base_url, job_urls[[j]], sep="")
                    
                    # read html
                    job_html_read <- read_html(full_job_url)

                    # extract company name
                    company_name <- job_html_read %>%
                        html_nodes(css= 'head > meta:nth-child(15)') %>%
                        html_attr('content')

                    # extract title
                    job_title <- job_html_read %>%
                        html_node('h1') %>%
                        html_text()
                    
                    time_since_placed_text <- job_html_read %>%
                        html_nodes(css= '.jobsearch-JobMetadataFooter > div:nth-child(2)') %>%
                        html_text()

                    # check if string does not contain integer using regex
                    if (!grepl("\\d", time_since_placed_text) & !grepl("Vandaag", time_since_placed_text)){
                        time_since_placed_text <- job_html_read %>%
                            html_nodes(css= '.jobsearch-JobMetadataFooter > div:nth-child(1)') %>%
                            html_text()
                    }

                    if (grepl("\\d", time_since_placed_text)){
                        time_since_placed <- str_extract_all(time_since_placed_text, "\\(?[0-9,.]+\\)?")[[1]]
                    } else if (grepl("Vandaag", time_since_placed_text)) {
                        time_since_placed <- 0
                    } else {
                    time_since_placed <- NA
                    }


                    # extract description
                    job_description <- job_html_read %>%
                        html_nodes(xpath= '//*[@id="jobDescriptionText"]') %>%
                        html_text()
                    
                    # remove non alpha numeric characters
                    job_description_cleaned <- gsub('[^[:alnum:] ]', ' ', job_description)

                    # generate wordvector
                    wordvec <- unname(unlist(sapply(job_description_cleaned, function(z) str_split(tolower(z), " "))))

                    # extract skills from word vector
                    skill_table <- wordvec[wordvec %in% tolower(wordlist)] %>%
                        unique()

                    # probeer salari op te halen via specifieke salary indicator
                    salary_indicator <- job_html_read %>%
                        html_nodes(css= '#salaryInfoAndJobType') %>%
                        html_text()

                    # Als de € niet voorkomt in het bestand probeer dan de salary distribution
                    if (length(salary_indicator) < 1){
                        salary_indicator <- job_html_read %>%
                            html_nodes(css= '.cmp-SalaryDistributionDisplayWidget-subinfo') %>%
                            html_text()
                    }

                    if (length(salary_indicator) < 1){
                        salary <- NA
                    } else if (grepl('€',salary_indicator)) {
                        # anders zoek de eerste salaris indicatie
                        salary_start_num <- unlist(gregexpr('€', salary_indicator))[1]
                        salary <- substring(salary_indicator, salary_start_num, salary_start_num + 7) 
                    } else {
                        salary <- NA
                    }

                    new_row <- c(
                        company_name,
                        job_title,
                        salary,
                        toString(unique(skill_table)),
                        as.numeric(time_since_placed),
                        job_description_cleaned,
                        full_job_url)
        
                    jobs_df[nrow(jobs_df) + 1,] = new_row
                }
            },
            warning = function(w){
                message("a warning occured with url. ", full_job_url)
                message("the data trying to be stored in the the dataframe is:")
                message(new_row[1])
                message(new_row[2])
                message(new_row[3])
                message(new_row[4])
                message(new_row[5])
                message(new_row[6])
                message(new_row[7])
                message(new_row[8])
                message("And below is the warning message from R:")
                message(w)

            },
            error=function(error_message) {
                message("an error occured with url.", full_job_url)
                message("And below is the error message from R:")
                message(error_message)
            },
            finally = {
                Sys.sleep(4)
            }
        )
    }
}

In [13]:
jobs_df

Company                                     
1  ING Netherlands                             
2  a.s.r.                                      
3  Gemeente Deventer                           
4  Data Consulting Group                       
5  Experis iSense                              
6  belastingdienst                             
7  Mploy Associates                            
8  APG                                         
9  Lightyear                                   
10 Deloitte                                    
11 Vlisco Netherlands                          
12 5CA                                         
13 World of Content                            
14 Bertelsmann SE & Co. KGaA - Corporate Center
15 Bol.com                                     
16 YoungCapital                                
17 YoungCapital                                
18 HAYS                                        
19 Gemeente Deventer                           
20 Data Consulting Group                       
21 APG                                         
22 Lightyear                                   
23 Deloitte                                    
24 Matchd                                      
25 5CA                                         
26 World of Content                            
27 Bol.com                                     
28 IG&H                                        
29 Catawiki                                    
30 EyeOn                                       
   job_title                                                salary  
1  Data Analyst                                             NA      
2  Informatie & Data Analist Risk & Beleggingen Dimension   \2004.500 -
3  Junior Data-analist / Onderzoeker                        \2002.865 -
4  TRAINEESHIP: DATA SCIENCE & ANALYTICS EXCELLENCE PROGRAM \2002.500 -
5  Junior Python Developer | Data Scientist                 \2003.500 -
6  Data-analist                                             \2005.139 p
7  Data Expert | Talent Program                             \2002.900 -
8  Traineeship Digitalization & Data Analytics              \200 2.826 
9  Data Scientist Performance Modeling                      NA      
10 Werkstudent Data Analytics                               NA      
11 Process and Data Specialist                              NA      
12 Junior Data Analyst in EMEA - SHR0267                    NA      
13 Data Scientist                                           \2003.000 -
14 Data Science Graduate Program                            NA      
15 Head of Data FinTech                                     \2006.750 -
16 Data verwerker                                           \2002.500 p
17 Data coördinator                                         \2002.205 -
18 Item Data Specialist                                     NA      
19 Junior Data-analist / Onderzoeker                        \2002.865 -
20 TRAINEESHIP: DATA SCIENCE & ANALYTICS EXCELLENCE PROGRAM \2002.500 -
21 Traineeship Digitalization & Data Analytics              \200 2.826 
22 Data Scientist Performance Modeling                      NA      
23 Werkstudent Data Analytics                               NA      
24 IV Expert Data en Business Intelligence                  NA      
25 Junior Data Analyst in EMEA - SHR0267                    NA      
26 Data Scientist                                           \2003.000 -
27 Head of Data FinTech                                     \2006.750 -
28 (Junior) Data Scientist                                  NA      
29 Data Scientist - Insights                                \200 4.183 
30 Data analyst planning and forecasting                    NA      
   skills                                                                                 
1  sql, sas, python                                                                       
2  r, python, scala                                                                       
3  spss                                             